In [ ]:
# 数据仓库任务一：字段统计注释

## 01张伟

In [ ]:
cur = conn.cursor()
cur.execute('select * from tls201_appln');
rows = cur.fetchall()
df = pd.DataFrame( [[ij for ij in i] for i in rows] )
#数量统计 均值 方差等
df.describe()

#是否存在null
df.notnull().sum()
df.isnull().sum()

#频次以及频率统计
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)
a=pd.value_counts(df.appln_kind)
print(a/a.sum())
print(a)

## 03刘慧

In [ ]:
table_a=pd.read_csv('tls201_appln-1.csv') #从CSV读入数据

## 04林达滔

In [ ]:
#有年份和日期的数据只有年份的数据
date=['appln_filing_date']#此处可填充其他同类型数据名
for i in date:
    day=tls201[i].copy()
    new=old=day[0]
    for b in range(1,day.shape[0]-1):
        if(day[b].year!=9999):
            if(day[b]>new):
                new=day[b];
            if(day[b]<=old):
                old=day[b];
    print('最新的',i,new)
    print('最早的',i,old)
    print('————————————————————')

该程序运行效率过低，可以考虑改进

In [ ]:
pd.set_option('display.float_format',lambda x : '%.2f' % x)
#由于pandas在输出数字时会使用科学计数法，故而此处设置一下关闭科学计数法，有小数时保留小数点后两位

该方法较好，可以控制全局输出的小数点。

## 05林可菁

In [ ]:
date=appln["EARLIEST_FILING_DATE"].tolist()
date.remove(date[0])
list=[]
for i in date:
    if i != 0:
        list.append(i)
print('最早日期：'+ str(min(list))+',最晚日期'+ str(max(list)))

最早日期：1900-01-01,最晚日期1900-12-20 这个方法没有控制好 9999-12-31

## 06彭宇晖

In [ ]:
def Extract(sql2):
    # 以tls201_appln表的国别appln_auth为例。
    # 首先提取数据
    cur=conn.cursor()
    cur.execute(sql2)
    LIST11=[] #将取出的数据放在一个List之中
    while 1:
        res=cur.fetchone()
        if res is None:
            #表示已经取完结果集
            break
        LIST11.append(res[0]) 

    cur.close()
    conn.commit()
    conn.close()
    return LIST11

A=[]
B=[]
C=[]

for i in list:
    sql2="select "+i+" from tls201_appln"
    TYPE=type(Extract(sql2)[0])
    if str(TYPE)=="<class 'str'>":
        A.append(i)
    if str(TYPE)=="<class 'int'>":
        B.append(i)
    if str(TYPE)=="<class 'datetime.datetime'>":
        C.append(i)

整体思路，迭代根据不同类型数据进行遍历，此思路较好。
最后时间分类应该改为'datetime.date'。
建议可以后续代码可以借鉴`07陈秋颖`、`23李昊璟`等同学的思路，可以针对表中的全部字段逐一展开分析，而不需要具体指定字段。
下面一种方法也可以快速调用所有表、字段、类型，可供参考。

In [ ]:
sql1 = """SELECT TABLE_NAME,COLUMN_NAME,DATA_TYPE
FROM information_schema.COLUMNS
WHERE TABLE_SCHEMA ='patstat2018bsample'"""

## 07陈秋颖

In [ ]:
#导入需要的包
library(RMySQL)

#设置语言运行环境
Sys.setlocale('LC_ALL', 'C')

#建立MySQL连接
con <- dbConnect(MySQL(), host = "127.0.0.1", dbname = "patstat2018b",  username = "root",  password = "*****")
#查看数据库patstat2018b中的表格
dbListTables(con)

#创建函数 analysis函数
analysis <- function(data){
  cat("\n\n", tables[i], " statistical results:", "\n")
  
  #查看表格的行数和列数
  dim(data)
  #记录表格行数
  n <- nrow(data)
  
  #统计表格中各属性的数据类型
  type <- cbind(colnames(data), sapply(data, function(y){class(y)}))
  #记录名字含有date的属性
  date <- grep("date", type[, 1])
  
  #将关于date的数据类型转变为Date
  if(length(date)!= 0){
    for(j in 1:length(date)){
      data[, date[j]] <- as.Date(data[, date[j]])
    }
  }
  
  #重新统计表格中各属性的数据类型
  type <- cbind(colnames(data), sapply(data, function(y){class(y)}))
  
  #统计ID型数据、申请号（nr）数据、分类型数据（文本型数据）、数量型数据
  id <- grep("id", type[, 1])
  nr <- grep("nr", type[, 1])
  char <- grep("character", type[, 2])
  char1 <- char[which(char %in% nr == FALSE)]
  integer <- grep("integer", type[, 2])
  integer <- integer[which(integer %in% id == FALSE)]
  
  #判断表格是否含有分类型数据
  if(length(char1) != 0){
    cat("\n\n", tables[i], "--statistical results of classified data:", "\n")
    
    #创建列表list1
    list1 <- list()
    
    for(j in 1:length(char1)){
      
      #统计分类型数据频次
      freq <- as.data.frame(table(data[, type[char1[j]]]))
      #计算频率并合并表格
      freq <- cbind(freq, (freq$Freq)/n)
      #重命名列名
      colnames(freq) <- c(colnames(data[char1[j]]), "Freq", "Rate")
      
      #多分类如果超过20类，则仅统计前20项
      list1[[j]] <- head(freq[order(-freq[, 2]), ], 20)
    }
    #展示分类型数据统计结果
    print(list1)
    
  }else{
    #如果表格不含有分类型数据，则加以说明
    cat(tables[i], "does not exist classified data other than application numbers", "\n\n")
  }
  
  #判断表格是否含有数量型数据
  if(length(integer) != 0){
    cat("\n\n", tables[i], "--statistical results of numeric data:", "\n")
    
    #创建列表list2
    list2 <- list()
    
    for(j in 1:length(integer)){
      
      #读入数量型数据
      intdata <- data[, type[integer[j]]]
      #分别统计最大值、最小值、平均值、方差
      list2[[j]] <- c(type[integer[j]], max = max(intdata), min = min(intdata), mean = round(mean(intdata), 3), var = round(var(intdata), 3))
    }
    #展示数量型数据统计结果
    print(list2)
    
  }else{
    #如果表格不含有数量型数据，则加以说明
    cat(tables[i], "does not exist numeric data other than ID type", "\n\n")
  }
  
  #判断表格是否含有日期型数据
  if(length(date) != 0){
    cat("\n\n", tables[i], "--statistical results of date data:", "\n")
    
    #创建列表list3
    list3 <- list()
    
    for(j in 1:length(date)){
      
      #读入日期型数据（排除"9999-12-31"的日期型数据）
      realdate <- data[which(data[, date[j]]!= "9999-12-31"), date[j]]
      #分别统计最早日期、最晚日期
      list3[[j]] <- c(type[date[j]], min = as.character(min(realdate)), max = as.character(max(realdate)))
    }
    #展示日期型数据统计结果
    print(list3)
    
  }else{
    #如果表格不含有日期型数据，则加以说明
    cat(tables[i], "does not exist date data", "\n\n")
  }
  
  #判断表格是否含有文本型数据
  if(length(char) != 0){
    cat("\n\n", tables[i], "--statistical results of text data:", "\n")
    
    #创建列表list4
    list4 <- list()
    
    for(j in 1:length(char)){
      
      #读入文本型数据
      chardata <- data[, type[char[j]]]
      #统计文本型数据的字段长度
      nchardata <- nchar(chardata)
      #分别统计字段平均长度、长度最小值、长度最大值
      list4[[j]] <- c(type[char[j]], mean = round(mean(nchardata), 3), min = min(nchardata), max = max(nchardata))
    }
    #展示文本型数据统计结果
    print(list4)
    
  }else{
    #如果表格不含有文本型数据，则加以说明
    cat(tables[i], "does not exist text data", "\n\n")
  }
}

#读入数据库patstat2018b中的表格
tables <- dbListTables(con)

#分别对表格进行表统计与字段统计
for(i in 1:length(tables)){
  data <- dbReadTable(con, tables[i])
  analysis(data)
}

非常欣赏这段代码，列出来供大家的参考。

## 08伍忠洋

```mysql
/*2.数量型数据：描述性统计（包括，最大值、最小值、平均值、方差）。*/
SELECT max(techn_field_nr) maxfield,MIN(techn_field_nr) minfield, avg(techn_field_nr) avgfield, STD(techn_field_nr) stdfield from tls230_appln_techn_field;

/*3.日期型数据：输出最早、最晚日期，最晚不得是9999-12-31。*/
SELECT MAX(appln_filing_date) maxdata, MIN(appln_filing_date) mindata from tls201_appln where appln_filing_date <'3000-1-1';

/*4.字段类型（文本型）：统计字段平均长度、长度最小值、长度最大值。*/
SELECT MAX(CHAR_LENGTH(continent)) max_char_length, MIN(CHAR_LENGTH(continent)) min_char_length, AVG(CHAR_LENGTH(continent)) avg_char_length FROM tls801_country;
```

纯用SQL语言操作也是一种思路。

## 09杨秋颖

In [ ]:
max=min=data[0]
for i in range(1,data.shape[0]-1):
    if(data[i].year!=9999):
        if(data[i]>max):
            max=data[i];
        elif(data[i]<min):
            min=data[i];

对四类不同类型的数据进行了分析。上述日期的处理方法不错，但整体结果呈现方面不统一。

## 10王可欣

利用R语言实现全部的分析，非常优秀。未来可以考虑用`tidyverse`风格的包来写脚本，这样输出的效果、代码的一致性会提升。

## 11杨洁

In [ ]:
select avg(nb_applicants) from ( #最外层开始
   select appln_id, nb_applicants from ( #第二层开始
         select appln_id, @index:=@index+1 as myindex, nb_applicants from tls201_appln, (select @index:=0) as initvar order by nb_applicants #最内层
    ) as t where floor(@index/2+1)=myindex or ceil(@index/2)=myindex #第二层结束
) as x  #最外层结束

`11杨洁`同学计算了中位数。MYSQL没有直接计算中位数的函数。

## 12杨贺

In [ ]:
table201['APPLN_NR'].sum() ##？？？

In [ ]:
table209['IPC_GENER_AUTH'].value_counts()[:20]/table209['IPC_GENER_AUTH'].value_counts()[:20].sum()

第二段代码原本我们是要求每个类表占全部数量的百分比，而不是前20类的百分比。

In [ ]:
APPLN_FILING_DATE_max=table201['APPLN_FILING_DATE'].max()
APPLN_FILING_DATE_min=table201['APPLN_FILING_DATE'].min()
print("APPLN_FILING_DATE最早最晚分别为",APPLN_FILING_DATE_min,APPLN_FILING_DATE_max)
# APPLN_FILING_DATE最早最晚分别为 2001-01-01 2031-12-01

结果可能有问题。

## 13代罡松

In [ ]:
df3=df1["appln_auth"].value_counts()
df4=df2
result=pd.concat([df3,df4],axis=1)
result.head(20)

In [ ]:
# 利用pd.concat使得打印效果会好，但列名则均为“appln_auth”，这个效果并不好，可以考虑改进。

## 14孙晨煜

用KNIME实现，但这样这样缺乏自动化的过程。可以考虑更自动化的来统计字段信息。

## 15向娇阳

用SPSS实现。非常细致。

## 16付初

In [ ]:
#输出最大日期和最小日期
q=tls201.appln_filing_date
q=q.replace('9999-12-31','2000-00-00')#替换成一个居中的日期，这样既不会影响最大日期也不会影响最小日期
d=tuple(q.sort_values())
print("最大日期是：",d[len(d)-1])
print("最小日期是：",d[0])

是一种思路，在你给出的单个字段的处理中或许没有问题，但如果是采用替换的方法，会在一定程度上改变日期的统计分布。
推荐`25金涛`同学日期的处理方法。

## 17陈月

很巧妙的利用了SQL语句来提取数据，这样可以减少数据IO的量。

In [ ]:
#设置平均数函数
def table_avg(table,column):
    sql='SELECT AVG(%s) FROM %s' % (column,table)
    cursor.execute(sql)
    result=cursor.fetchone()
    df = pd.DataFrame({column:[column],'avg':[result]})
    print(df)
    
table_avg('tls901_techn_field_ipc','techn_field_nr')

上述写法`%(column,table)`的例子要谨慎使用，它涉及到了MYSQL防注入的问题。虽然，本例子中你不涉及。
具体可以参见以下例子：

In [ ]:
#! /usr/bin/env python
# -*- coding:utf-8 -*-
import pymysql
 
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='', db='User')
cursor = conn.cursor()
username= input()
password =input()

#执行参数化查询
row_count=cursor.execute("select user,pwd from User where user='%s' and pwd='%s'" ,(username,password))
#execute()函数本身就有接受SQL语句变量的参数位，只要正确的使用（直白一点就是：使用”逗号”，而不是”百分号”）就可以对传入的值进行correctly转义，从而避免SQL注入的发生。


#内部执行参数化生成的SQL语句，对特殊字符进行了加\转义，避免注入语句生成。
# sql=cursor.mogrify("select user,pwd from User where user='%s' and pwd='%s'" ,(username,password))
# print (sql)
 
row_1 = cursor.fetchone()
print(row_count,row_1)
 
conn.commit()
cursor.close()
conn.close()

当输入了上面的用户名和密码，服务端的sql就变成：
username：` ‘or 1 = 1 –-`
password：``
```mysql
sql = "select user,pwd from User where user=‘'or 1 = 1 –-' and pwd='%s'"
```
因为条件后面username=”or 1=1 用户名等于 ” 或1=1 那么这个条件一定会成功；然后后面加两个-，这意味着注释，它将后面的语句注释，让他们不起作用，这样语句永远都能正确执行，用户轻易骗过系统，获取合法身份。

第二点，目前来看，这种拼接参数的方法，似乎每次只计算一次如，仅计算AVG，似乎效率有点低。是否有更高效的同时计算多个值方法。

In [ ]:
## 18余欣

In [ ]:
很标准的SQL语句，对数值型、分类型、日期型的数据分别进行了分析

In [ ]:
## 19王雪蓉

(d)日期型数据：输出最早、最晚日期，最晚不得是9999-12-31,已经在describe中进行了比较。

但实际输出结果，如‘EARLIEST_FILING_YEAR’并没有排除‘9999’，因此，需要注意。还有如果在整体数据中排出‘9999’实际是一个删除操作，就是删掉所有时间为‘9999’的专利，这样如果是统计数量则可能与其他同学的存在差异。

In [ ]:
## 20冯泽宇

与前面同学做的PYTHON脚步类似。建议结果能有更规范化，或者是表格式输出。

## 21高爽

`21高爽`同学做的SQL语句非常细致。这里使列举一个例子和`18余欣`同学的语句进行比较

```mysql
#  21高爽同学代码
select ipc_version, nr_ipc_version, concat(round(nr_ipc_version / total*100, 4),'%') ratio from(
   select * from (select ipc_version, count(1) nr_ipc_version from tls209_appln_ipc
   group by ipc_version
   order by nr_ipc_version desc
   limit 20 ) temp1
   inner join
   (select count(1) total from tls209_appln_ipc) temp2
   on 1=1
) temp3
```
```mysql
# 18余欣同学代码
/*39.ipc_version 日期型数据*/
SELECT
    MIN( ipc_version ) AS 最早日期,
    MAX( ipc_version ) AS 最晚日期 
FROM
    tls209_appln_ipc
WHERE
    ipc_version <> '9999-12-31';
```

上面代码的最大区别在于两位同学对`ipc_version`这个字段类型的理解，虽然，两种方法并无对错之别，`ipc_version`的确也是一个日期类型的数据，但实际上其含义更倾向于一个说明书的版本号。因此，我个人觉得`21高爽`作为分类进行统计的方法可能更适合。

## 22郭耀元

以下是`22郭耀元`对非空值的计算方法，与`df.isnull().sum()`方法稍有不同。

In [ ]:
df201.shape[0]-df201.count(axis=0)

## 23李昊璟

`23李昊璟`同学的思路非常好。首先是采用了`mysql.connector`包，其次，他用了四个非常简洁的函数来表达四种不同类型的统计：
分别是：CalNUM、Calcate、CalText、CalDate

In [ ]:
def CalNUM(arr):
    try:
        print("平均值为："+str(arr.mean()))
        print("方差为:"+str(arr.var(ddof=0)))
        print("最大值为："+str(arr.max()))
        print("最小值为："+str(arr.min()))
    except:
        print("检查数据类型")

In [ ]:
def CalDate(arr):
    latestdate=arr[0]
    earliestdate=arr[0]
    for i in range(1,len(arr)):
        if arr[i]!=datetime.date(9999,12,31):
            if arr[i]>latestdate:
                latestdate=arr[i]
            if arr[i]<earliestdate:
                earliestdate=arr[i]
    return earliestdate,latestdate

In [ ]:
def CalText(arr):
    TextLength=len(arr[0])
    LongestLength=len(arr[0])
    ShortestLength=len(arr[0])
    for i in range(1,len(arr)):
        TextLength=TextLength+len(arr[i])
        if len(arr[i])<ShortestLength:
            ShortestLength=len(arr[i])
        if len(arr[i])>LongestLength:
            LongestLength=len(arr[i])      
    return LongestLength,ShortestLength,TextLength/len(arr)

In [ ]:
def Calcate(arr):
    counts = {}
    for word in arr:
        counts[word] = counts.get(word, 0) + 1
        items = list(counts.items())
        items.sort(key=lambda x: x[1], reverse=True)
    for i in range(min(20,len(items))):   #输出前20
        word, count = items[i]
        print(str(word),str(count),str(count/len(arr)))

当然，上述代码也有改进之处，例如可以结合`25金涛`同学的代码使整个过程更流畅。例如，针对CalNUM

In [ ]:
def sql_execute(sql):
    mycursor= mydb.cursor()
    mycursor.execute(sql)
    myresult = mycursor.fetchall() 
    cols = mycursor.description
    col = []
    for i in cols:
        col.append(i[0])
        myresult=list(map(list,myresult))
    myresult = pd.DataFrame(myresult,columns=col) 
    return myresult 

def CalNUM(arr):
    try:
        print("平均值为："+str(arr.mean()))
        print("方差为:"+str(arr.var(ddof=0)))
        print("最大值为："+str(arr.max()))
        print("最小值为："+str(arr.min()))
    except:
        print("检查数据类型")

for i in range(0,data.shape[1]):
    print(data.keys()[i],"\n")
    print(CalNUM(data.iloc[:,i].dropna()),"\n")

当然，最好还是能够`07陈秋颖`同学的代码那样，将各部分的各种类型组合起来统一输出。

## 24尹梦瑶

利用SQL对全部字段进行了逐一的描述统计。

## 25金涛

In [ ]:
#tls201 分类型数据
sql201_1 = "SELECT tls201_appln.appln_auth,tls201_appln.appln_kind,tls201_appln.int_phase,tls201_appln.reg_phase,tls201_appln.nat_phase FROM tls201_appln"
cursor.execute(sql201_1)
results201_1 = cursor.fetchall()
cols = cursor.description
col = []
for i in cols:
    col.append(i[0])
    results201_1=list(map(list,results201_1))
results201_1 = pd.DataFrame(results201_1,columns=col) #转换成dataframe

for i in range(0,results201_1.shape[1]): 
    print(results201_1.iloc[:,i].value_counts().head(20)) #频次
    print("\n")
    print((results201_1.iloc[:,i].value_counts()/sum(results201_1.iloc[:,i].value_counts())).head(20)) #频次百分比
    print("\n")

In [ ]:
#tls201 日期型数据
sql201_3 = "SELECT tls201_appln.appln_filing_date,tls201_appln.earliest_filing_date,tls201_appln.earliest_publn_date FROM tls201_appln"
cursor.execute(sql201_3)
results201_3 = cursor.fetchall()
cols = cursor.description
col = []
for i in cols:
    col.append(i[0])
    results201_3=list(map(list,results201_3))
results201_3 = pd.DataFrame(results201_3,columns=col) #转换成dataframe
max=results201_3.apply(lambda x: x.sort_values().unique()[-2], axis=0)
min=results201_3.apply(lambda x: x.sort_values().unique()[0], axis=0)
print(max)
print(min)

思路很清晰，逻辑也很不错。

## 26陈俊璇

利用SQL对全部字段进行了逐一的描述统计。

## 27潘倩月

利用KNIME实现对全部字段的分析。
有以下几个小问题可以注意：（1）时间‘9999’没有排除，至少在201表中。

In [ ]:
## 28谭悦

In [ ]:
利用KNIME实现对全部字段的分析。
# 勾选了Median取值；
# 多分类超过20类仅统计Top/Bottom 20；
# 根据日期型数据数量将最大数量调整为30,000（实际为27,900左右）
实际上在做分类数据的描述性统计时，还有
问题：
似乎也没有注意时间‘9999’没有排除。

## 29张隽若

用STATA实现对全部字段的分析。
STATA、SPSS等工具对数据进行描述统计还是非常方便的，统计指标也很完整。
`29张隽若`也链接了数据库。

## 30周雨涵

In [ ]:
```mysql
# 原始代码
/*（b）数量型数据：描述性统计（包括，最大值、最小值、平均值、方差）。*/
create table t2(
  max_value int unsigned not null,
	min_value int unsigned not null,
	avg_value int unsigned not null      
);engine=innodb default charset=utf8

std_value int unsigned not null;
select max(appln_id), min(appln_id), AVG(DISTINCT appln_id), STD(appln_id) 
from tls201_appln 
as max_value, min_value, avg_value, std_value
```
# 好像运行不了

mysql中将检索结果创建到新表有两种方法：
```mysql
# 新表不存在
create table new_table
select * from old_talbe;
```
```mysql
# 新表存在
insert into new_table(field1,field2,.....)
select * from old_table;
```
另外，通常对于分析、统计而言，建立视图也是很好的思路。